# Timeseries Analysis and Forcasting

### Importing Data

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn.objects as so
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

timeseries_df = pd.read_csv('../data/processed_data/timeseries.csv')

In [23]:
timeseries_df['price_sum'] = timeseries_df['quantity'] * timeseries_df['price']

In [24]:
timeseries_df

,customer_id,date,quantity,price,month,year,dayofweek,day,price_sum
0,1,1997-01-01,1,11.77,January,1997,2,Wednesday,11.77
1,2,1997-01-12,1,12.00,January,1997,6,Sunday,12.00
2,2,1997-01-12,5,77.00,January,1997,6,Sunday,385.00
3,3,1997-01-02,2,20.76,January,1997,3,Thursday,41.52
4,3,1997-03-30,2,20.76,March,1997,6,Sunday,41.52
...,...,...,...,...,...,...,...,...,...
69654,23568,1997-04-05,4,83.74,April,1997,5,Saturday,334.96
69655,23568,1997-04-22,1,14.99,April,1997,1,Tuesday,14.99
69656,23569,1997-03-25,2,25.74,March,1997,1,Tuesday,51.48
69657,23570,1997-03-25,3,51.12,March,1997,1,Tuesday,153.36


### Total Spend over Time

In [25]:
timeseries_agg = timeseries_df.groupby('date').agg({'price_sum': 'sum'}).reset_index()

fig = px.line(timeseries_agg, 
              x='date', 
              y='price_sum', 
              title='Total Spend Over Time',
              labels={'price_sum': 'Total Spend', 'date': 'Date'})
fig.write_image('../visuals/total_spend_over_time.png')
fig.show()


### Avg Spend over Month and DayofWeek

In [26]:
month_avg = timeseries_df.groupby('month').agg({'price_sum': 'mean'}).reset_index()
dayofweek_avg = timeseries_df.groupby('dayofweek').agg({'price_sum': 'mean'}).reset_index()

fig1 = px.bar(month_avg, 
              x='month', 
              y='price_sum', 
              title='Average Spend by Month',
              labels={'price_sum': 'Average Spend', 'month': 'Month'},
              color='month')

fig2 = px.bar(dayofweek_avg, 
              x='dayofweek', 
              y='price_sum', 
              title='Average Spend by Day of the Week',
              labels={'price_sum': 'Average Spend', 'dayofweek': 'Day of Week'},
              color='dayofweek')

fig1.write_image('../visuals/avg_spend_by_month.png')
fig2.write_image('../visuals/avg_spend_by_dayofweek.png')

fig1.show()
fig2.show()


### Revenue with Moving Averages

In [27]:
timeseries_agg['7-day MA'] = timeseries_agg['price_sum'].rolling(window=7).mean()
timeseries_agg['30-day MA'] = timeseries_agg['price_sum'].rolling(window=30).mean()

fig = px.line(timeseries_agg, 
              x='date', 
              y=['price_sum', '7-day MA', '30-day MA'], 
              title='Total Spend with Moving Averages',
              labels={'value': 'Spend', 'variable': 'Type', 'date': 'Date'})

fig.write_image('../visuals/total_spend_with_moving_avg.png')
fig.show()


### Customer Spend over Time Density Heatmap

In [28]:
heatmap_data = timeseries_df.pivot_table(index='dayofweek', columns='month', values='price_sum', aggfunc='sum')

fig = px.imshow(heatmap_data, 
                labels=dict(x="Month", y="Day of Week", color="Total Spend"), 
                x=heatmap_data.columns, 
                y=heatmap_data.index,
                title='Heatmap of Customer Spend by Month and Day of Week')
fig.write_image('../visuals/customer_spend_over_time_heatmap.png')
fig.show()


### Spend Forecast for the Next 90 days

In [29]:
from prophet import Prophet

# Prepare data for Prophet
forecast_df = timeseries_agg.rename(columns={'date': 'ds', 'price_sum': 'y'})

# Fit the model
model = Prophet()
model.fit(forecast_df)

# Create future dates
future = model.make_future_dataframe(periods=90)
forecast = model.predict(future)

# Plot the forecast
fig = px.line(forecast, 
              x='ds', 
              y=['yhat', 'yhat_lower', 'yhat_upper'], 
              title='Spend Forecast for the Next 90 Days',
              labels={'ds': 'Date', 'value': 'Predicted Spend'})
fig.write_image('../visuals/spend_forecast_for_next_90_days.png')
fig.show()


20:00:23 - cmdstanpy - INFO - Chain [1] start processing
20:00:23 - cmdstanpy - INFO - Chain [1] done processing
